# 🧠 Урок 27: Transfer Learning и предобученные модели
**Цель урока:** Понять концепцию Transfer Learning, научиться использовать предобученные модели (например, MobileNet, ResNet) для решения новых задач. Подходит для новичков.

## 📌 Что такое Transfer Learning?
- **Transfer Learning** — это подход, при котором модель, обученная на одной задаче, используется в качестве основы для другой задачи.
- **Зачем?** Экономит время, ресурсы и позволяет достигать высокой точности на малых датасетах.
- **Аналогия:** Представьте, что вы учитесь играть на гитаре. Вместо обучения с нуля, вы используете навыки игры на пианино (похожие закономерности) [[1]].

## 📐 Почему используют предобученные модели?
- **Предобученные модели** уже изучили признаки на огромных датасетах (например, ImageNet).
- **Преимущества:**
  - Не нужно обучать сеть с нуля (экономия времени).
  - Высокая точность на новых задачах.
  - Меньше требований к данным (работает даже на 100–1000 изображений).
- **Популярные архитектуры:**
  - **ResNet:** Для сложных задач, устойчив к переобучению.
  - **MobileNet:** Легковесная модель для мобильных устройств.
  - **VGG16:** Простая и понятная для новичков.
- **Аналогия:** Предобученная модель — как готовый рецепт, который вы адаптируете под свои продукты [[1]].

## 🛠️ Как работает Transfer Learning?
### 1. Фризинг (заморозка) слоев
- **Что это?** Зафиксировать веса старых слоев, чтобы они не обновлялись при обучении.
- **Зачем?** Сохранить изученные признаки (например, края, формы) для новой задачи.
- **Пример:**
  ```python
  base_model.trainable = False  # Заморозка всей базовой модели
  ```
- **Аналогия:** Вы используете чужой ноутбук, но не меняете его код, только добавляете свой скрипт.

### 2. Fine-tuning (дообучение)
- **Что это?** Разморозка части слоев и дообучение на новых данных.
- **Зачем?** Адаптация модели к специфичным признакам вашей задачи.
- **Пример:**
  ```python
  for layer in base_model.layers[-20:]:  # Разморозка последних 20 слоев
      layer.trainable = True
  ```
- **Аналогия:** После использования рецепта, вы экспериментируете с ингредиентами, чтобы улучшить блюдо.

## 📦 Практика: Transfer Learning на датасете котов и собак
### Шаг 1: Загрузка данных

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Путь к данным
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip '
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# Генераторы данных
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

### Шаг 2: Загрузка предобученной модели

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models

# Загрузка MobileNet без верхнего слоя
base_model = MobileNet(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

# Добавление новых слоев
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(base_model.input, output)

### Шаг 3: Фризинг слоев и обучение

In [ ]:
# Фризинг
base_model.trainable = False

# Компиляция
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Обучение
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2)

### Шаг 4: Fine-tuning
```python
# Разморозка последних слоев
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Компиляция с малым learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Дообучение
history_fine = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2)
```

## 📊 Визуализация обучения
### Сравнение accuracy до и после fine-tuning

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy до и после fine-tuning')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss до и после fine-tuning')
plt.xlabel('Epochs')
plt.show()

## 📈 Преимущества Transfer Learning
- **Экономия времени:** Не нужно обучать сеть с нуля, можно использовать готовые признаки.
- **Малый датасет:** Работает даже с 100–1000 изображениями.
- **Высокая точность:** Предобученные модели уже изучили важные признаки (например, края, текстуры).
- **Гибкость:** Можно заменять верхние слои под свою задачу.
- **Аналогия:** Использование готового автомобиля для перевозки картошки вместо постройки с нуля.

## 🧪 Практика: Сравнение с обучением с нуля
### Шаг 1: Обучение с нуля

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Модель с нуля
scratch_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция и обучение
scratch_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

history_scratch = scratch_model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2)

### Шаг 2: Сравнение с Transfer Learning

In [ ]:
# Сравнение точности
plt.plot(history_scratch.history['val_accuracy'], label='Обучение с нуля')
plt.plot(history_fine.history['val_accuracy'], label='Transfer Learning + Fine-tuning')
plt.legend()
plt.title('Сравнение точности')
plt.xlabel('Эпохи')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## 🧠 Теория: Как устроены предобученные модели?
- **Низкоуровневые слои:** Обнаруживают края, текстуры, цвета.
- **Высокоуровневые слои:** Узнают сложные признаки (например, глаза, лапы у животных).
- **Где взять?** `tf.keras.applications` содержит VGG16, ResNet, MobileNet и другие [[1]].
- **Как адаптировать?**
  1. Удалите верхний слой (`include_top=False`).
  2. Добавьте свои слои для новой задачи.
  3. Фризинг и обучение.
  4. Fine-tuning.
- **Аналогия:** Предобученная модель — как шаблон одежды, который вы подгоняете под клиента.

## 📝 Домашнее задание
**Задача 1:** Измените архитектуру Transfer Learning:
- Добавьте `Dropout(0.5)` после `GlobalAveragePooling2D()`.
- Замените `MobileNet` на `VGG16`.
- Как меняется `val_accuracy`?
- Нарисуйте графики обучения.

**Задача 2:** Напишите отчет (200–300 слов), где:
- Опишите, как вы адаптировали модель.
- Объясните, почему Transfer Learning работает лучше, чем обучение с нуля.
- Приведите примеры, где Transfer Learning особенно полезен (например, медицинская диагностика с малым количеством данных).
- Оцените, как `Dropout` влияет на переобучение.

In [ ]:
# Ваш код здесь
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Загрузка VGG16
base_model = VGG16(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

# Добавление новых слоев с Dropout
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(base_model.input, output)

In [ ]:
# Обучение VGG16
base_model.trainable = False
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2)

In [ ]:
# Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-100]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2)

In [ ]:
# Визуализация обучения
import matplotlib.pyplot as plt

plt.plot(history.history['val_accuracy'], label='Validation Accuracy (freeze)')
plt.plot(history_fine.history['val_accuracy'], label='Validation Accuracy (fine-tuning)')
plt.legend()
plt.title('Accuracy до и после fine-tuning')
plt.xlabel('Эпохи')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## ✅ Рекомендации по выполнению домашнего задания
- **Задача 1:** Используйте `Dropout` для борьбы с переобучением.
- **Задача 2:** Сравнивайте `val_accuracy` и `val_loss` для разных архитектур.
- **Подсказка:** В `VGG16` больше слоев, поэтому fine-tuning требует большего числа эпох.